In [1]:
# Import necessary libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import least_squares

In [2]:
FILEPATH = 'ece661_pics\\hw9_image\\'
np.set_printoptions(precision=3)

In [3]:
world_cornors = np.array([
    [0, 0], [24, 0], [49, 0], [78, 0], [97, 0], [122, 0], [146, 0], [170, 0],
    [0, 24], [24, 24], [49, 24], [78, 24], [97, 24], [122, 24], [146, 24], [170, 24],
    [0, 49], [24, 49], [49, 49], [78, 49], [97, 49], [122, 49], [146, 49], [170, 49],
    [0, 78], [24, 78], [49, 78], [78, 78], [97, 78], [122, 78], [146, 78], [170, 78],
    [0, 97], [24, 97], [49, 97], [78, 97], [97, 97], [122, 97], [146, 97], [170, 97],
    [0, 122], [24, 122], [49, 122], [78, 122], [97, 122], [122, 122], [146, 122], [170, 122],
    [0, 146], [24, 146], [49, 146], [78, 146], [97, 146], [122, 146], [146, 146], [170, 146],
    [0, 170], [24, 170], [49, 170], [78, 170], [97, 170], [122, 170], [146, 170], [170, 170],
    [0, 194], [24, 194], [49, 194], [78, 194], [97, 194], [122, 194], [146, 194], [170, 194],
    [0, 219], [24, 219], [49, 219], [78, 219], [97, 219], [122, 219], [146, 219], [170, 219]
])

In [4]:
class Lines():
    '''
        Class for line from rho and theta.
    '''
    def __init__(self, rho, theta):
        self.rho = rho
        self.theta = theta
    
    def get_HC(self):
        '''
          Return homogenous coordinate of line.
        '''
        pt0 = np.array([self.rho*np.cos(self.theta), 
                        self.rho*np.sin(self.theta), 1])
        pt1 = pt0 + np.array([100*np.sin(self.theta), 
                              -100*np.cos(self.theta), 0])
        line_HC = np.cross(pt0, pt1)
        self.HC = line_HC / line_HC[2]

        return self.HC

In [5]:
class Image():
    ''' 
        Class for storing images.
    '''
    
    def __init__(self, path):
        self.path = path   
        self.load() 
        self.find_edge()
        self.find_line()
        self.get_corner()

    def load(self):
        filename = f'{self.path.parent}\\{self.path.name}'
        self.image = cv.imread(filename)
        self.image_gray = cv.cvtColor(self.image, cv.COLOR_BGR2GRAY)

    def show(self):
        plt.imshow(cv.cvtColor(self.image, cv.COLOR_BGR2RGB))
    
    def find_edge(self, min_val=150, max_val=255):
        ''' 
            Find edges using Canny algotirhm.
        '''
        blur_img = cv.GaussianBlur(self.image_gray, (5,5), 0)
        self.edge = cv.Canny(blur_img, min_val, max_val)
        filename = f'{self.path.parent}\\{self.path.stem}_edge.png'
        cv.imwrite(filename, self.edge)

        return self.edge

    def find_line(self, threshore=50):
        '''
            Find lines using Hough transformation.
         '''
        img = self.image.copy()
        # Find all possible lines
        lines = cv.HoughLines(self.edge, 1 , np.pi/180, threshore)

        # Separate horizontal lines and vertical lines
        thetas_H = []
        rhos_H = []
        thetas_V = []
        rhos_V = []
        for line in lines:
            rho, theta = line[0]
            if 0.25 < theta / np.pi < 0.75 :
                rhos_H.append(rho)
                thetas_H.append(theta)
            else:
                rhos_V.append(rho)
                thetas_V.append(theta)

        rhos_H = np.array(rhos_H)
        rhos_V = np.array(rhos_V)
        thetas_H = np.array(thetas_H)
        thetas_V = np.array(thetas_V)

        def filter_line(rhos, thetas, num_line):
            '''
                Filter lines. Keep only certain number.
            '''
            # Sort parameters
            idx = np.argsort(np.abs(rhos))
            rhos = rhos[idx]
            thetas = thetas[idx]
            # Keep running until certain number of lines left
            while rhos.size > num_line:
                diff = np.abs(np.diff(np.abs(rhos)))
                idx_diff_min = np.argwhere(diff == diff.min())[0][0]
                # Drop small difference parameter
                rhos = np.delete(rhos, idx_diff_min)
                thetas = np.delete(thetas, idx_diff_min)
            lines = []
            # Draw line
            for i in range(rhos.size):
                rho = rhos[i]
                theta = thetas[i]
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                cv.line(img, (x1, y1), (x2, y2), (0, 0, 255), 1)
                lines.append(Lines(rho, theta))

            return lines

        self.lines_H = filter_line(rhos_H, thetas_H, 10)
        self.lines_V = filter_line(rhos_V, thetas_V, 8)

        filename = f'{self.path.parent}\\{self.path.stem}_lines.png'
        cv.imwrite(filename, img)

    def get_corner(self):
        '''
            Compute corner from lines.
        '''
        img = self.image.copy()
        i = 0
        corners = []
        for line_H in self.lines_H:
            for line_V in self.lines_V:
                i += 1
                corner = np.cross(line_H.get_HC(), line_V.get_HC())
                corner = corner / corner[2]
                coordinate = corner[:2].astype(np.int)
                x = coordinate[0]
                y = coordinate[1]
                cv.circle(img, (x, y), 2, (0, 0, 255), 2)
                cv.putText(img, str(i), (x, y), 
                           cv.FONT_HERSHEY_SIMPLEX, 
                           0.5, (255, 255, 0), 1)
                corners.append([x, y])
            
        self.corners = np.array(corners)    
        filename = f'{self.path.parent}\\{self.path.stem}_corners.png'
        cv.imwrite(filename, img)

        # Compute Homography matrix
        A = np.zeros((2*len(corners), 9), np.float32)
        for i in range(len(corners)):
            x1 = world_cornors[i][0]
            y1 = world_cornors[i][1]
            x2 = corners[i][0]
            y2 = corners[i][1]
            A[2*i, :] = np.array([0, 0, 0, -x1, -y1, -1, x1*y2, y1*y2, y2])
            A[2*i+1, :] = np.array([x1, y1, 1, 0, 0, 0, -x1*x2, -y1*x2, -x2])
        u, s, v = np.linalg.svd(A)
        idx = np.argmin(s)
        eig_vector = v[idx, :]
        self.H = eig_vector.reshape((3 , 3))

    def compute_extrinsic(self, K):
        '''
            Compute extrincic parameters.
            K (intrinsic parameter) is requiered.
        '''
        h1 = self.H[:, 0]
        h2 = self.H[:, 1]
        h3 = self.H[:, 2]
        K_inv = np.linalg.inv(K)
        e = 1 / np.linalg.norm(np.dot(K_inv, h1))
        t = e * np.dot(K_inv, h3)
        r1 = e * np.dot(K_inv, h1)
        r2 = e * np.dot(K_inv, h2)
        r3 = np.cross(r1, r2)
        Q = np.array([r1, r2, r3]).T
        u, s, vt = np.linalg.svd(Q)
        R = np.dot(u, vt)
        # Rt = np.append(R, t.reshape(-1,1), axis=1)
        
        return R, t


In [6]:
def load_dataset(number=1):
    dataset = []
    directory = f'{FILEPATH}Dataset{number}'
    imgs = Path(directory).iterdir()
    for img in imgs:
        if img.suffix == '.jpg' or img.suffix == '.JPG':
            dataset.append(Image(img))
    
    return dataset

In [9]:
dataset = load_dataset(1)

In [850]:
def compute_V(H, i, j):
    ''' 
        Compute Vij from H
    '''
    hi = H[:, i-1]
    hj = H[:, j-1]
    Vij = np.zeros((6, 1))
    Vij[0] = hi[0] * hj[0]
    Vij[1] = hi[0] * hj[1] + hi[1] * hj[0]
    Vij[2] = hi[1] * hj[1]
    Vij[3] = hi[2] * hj[0] + hi[0] * hj[2]
    Vij[4] = hi[2] * hj[1] + hi[1] * hj[2]
    Vij[5] = hi[2] * hj[2]

    return Vij


In [1279]:
# Compute instinsic parameter
V = np.zeros((2*len(dataset), 6))
# Create V matrix 80 x 6
for i, img in enumerate(dataset):
    V[2*i, :] = compute_V(img.H, 1, 2).T
    V[2*i+1, :] = (compute_V(img.H, 1, 1) 
                   - compute_V(img.H, 2, 2)).T

# SVD decomposition
u, s, vt = np.linalg.svd(V)
b = vt[-1, :]
# Rename W
w11 = b[0]
w12 = b[1]
w22 = b[2]
w13 = b[3]
w23 = b[4]
w33 = b[5]
# Compute parameters
x0 = (w12*w13 - w11*w23) / (w11*w22 - w12**2)
l = w33 - (w13**2 + x0*(w12*w13 - w11*w23)) / w11
alpha_x = np.sqrt(l / w11)
alpha_y = np.sqrt((l*w11) / (w11*w22 - w12**2))
s = -(w12* (alpha_x**2) *  alpha_y) / l
y0 = s*x0/alpha_y - w13 * (alpha_x**2) / l

# Assembly K 
K = np.zeros((3, 3))
K[0, 0] = alpha_x
K[0, 1] = s
K[0, 2] = x0
K[1, 1] = alpha_y
K[1, 2] = y0
K[2, 2] = 1
print('K (intrinsic) paramenter before LM opimization \n', K)

K (intrinsic) paramenter before LM opimization 
 [[462.691  -9.861 265.43 ]
 [  0.    468.378 201.913]
 [  0.      0.      1.   ]]


In [1280]:
def build_params(K, dataset):
    '''
        Compack parameters into 1D vector.
        K Wx1 t1 Wx2 t2 Wx3 T3 ...
    '''
    params = np.array(K[0, :])
    params = np.append(params, K[1, 1:3])
    for img in dataset:
        R, t = img.compute_extrinsic(K)
        params = np.append(params, cv.Rodrigues(R)[0])
        params = np.append(params, t)
    return np.array(params)

In [1281]:
params = build_params(K, dataset)

In [1088]:
def extract_params(params):
    '''
        Invert of build params function.
    '''
    K = np.zeros((3, 3))
    K[0][0] = params[0]
    K[0][1] = params[1]
    K[0][2] = params[2]
    K[1][1] = params[3]
    K[1][2] = params[4]
    K[2][2] = 1
    params = np.delete(params, np.arange(0,5))
    ws = []
    ts = []
    while params.size > 0:
        ws.append(params[:3])
        params = np.delete(params, np.arange(0,3))
        ts.append(params[:3])
        params = np.delete(params, np.arange(0,3))

    return K, np.array(ws), np.array(ts)

In [1282]:
def cost_function(params, dataset, world_cornors):
    '''
        Residal function for LM optimization.
    '''
    error_vector = []
    K, ws, ts = extract_params(params)
    world_coordinate = np.hstack((world_cornors, 
                                  np.zeros((80, 1)), 
                                  np.ones((80, 1)))) 
    for i, img in enumerate(dataset):
        # Compute error for each image
        w = ws[i]
        t = ts[i]
        R = cv.Rodrigues(w)[0]
        Rt = np.append(R, t.reshape(-1,1), axis=1) 
        proj_coordinates = np.dot(np.dot(K, Rt), world_coordinate.T).T
        for j, corner in enumerate(img.corners):
            # Compute error on each point
            proj_coordinate = proj_coordinates[j]
            proj_coordinate /= proj_coordinate[2]
            proj_coordinate = proj_coordinate[:2]
        
            # Error on each point
            error_vector.append(np.linalg.norm(proj_coordinate - corner))

    return np.array(error_vector)

In [1283]:
sol = least_squares(cost_function, params, method='lm', 
                    args=(dataset, world_cornors),
                    max_nfev=5000)

In [1284]:
K_lm, ws_lm, ts_lm = extract_params(sol.x)
print('K (intrinsic) paramenter after LM opimization \n', K_lm)

K (intrinsic) paramenter after LM opimization 
 [[476.48   -6.829 192.642]
 [  0.    483.38  254.072]
 [  0.      0.      1.   ]]


In [1285]:
error_before = cost_function(params, dataset, world_cornors)
error_after = cost_function(sol.x, dataset, world_cornors)
print('Error \t Before LM \t After LM')
print(f'Mean \t {error_before.mean():8.3f} \t {error_after.mean():7.3f}')
print(f'Var \t {error_before.var():8.3f} \t {error_after.var():7.3f}')

Error 	 Before LM 	 After LM
Mean 	    6.942 	   3.053
Var 	   17.033 	   3.356


In [1186]:
def KRt2H(K, R, t):
    '''
        Compute Homography matrix from
        K (intinsic) and Rt (extrinsic).
    '''
    Rt = np.append(R, t.reshape(-1,1), axis=1)
    Rt = np.delete(Rt, 2, axis=1)
    H = np.dot(K, Rt)

    return H

In [1286]:
fix_image = 2 # Pic_11
# Compute Homography of fix image
R, t = dataset[fix_image].compute_extrinsic(K)
H_fix = KRt2H(K, R, t)
for img in dataset:
    i = 0
    base_img = dataset[fix_image].image.copy()
    base_corner = dataset[fix_image].corners
    # Draw fix image corners in red
    for corner in base_corner:
        x, y = corner
        i += 1
        cv.circle(base_img, (x, y), 2, (0, 0, 255), 2)
        cv.putText(base_img, str(i), (x, y), 
                    cv.FONT_HERSHEY_SIMPLEX, 
                    0.5, (255, 255, 0), 1)
    # Compute projected homogramphy matrix
    R, t = img.compute_extrinsic(K)
    H = KRt2H(K, R, t)
    # Corners to be projected
    corners = img.corners
    corners = np.hstack((corners, np.ones((80, 1))))
    proj_world = np.dot(np.linalg.inv(H), corners.T)
    proj_corner = np.dot(H_fix, proj_world).T
    for corner in proj_corner:
        x = int(corner[0] / corner[2])
        y = int(corner[1] / corner[2])
        # Draw projected corner in blue
        cv.circle(base_img, (x, y), 2, (255, 0, 0), 2)

    path = img.path
    filename = f'{path.parent}\\before_LM\\{path.stem}.png'
    cv.imwrite(filename, base_img)

In [1287]:
# Compute homography matrix of fix image after LM
w = ws_lm[fix_image]
R = cv.Rodrigues(w)[0]
t = ts_lm[fix_image]
H_fix_lm = KRt2H(K_lm, R, t)
for num, img in enumerate(dataset):
    i = 0
    base_img = dataset[fix_image].image.copy()
    base_corner = dataset[fix_image].corners
    # Draw base corner in red
    for corner in base_corner:
        x, y = corner
        i += 1
        cv.circle(base_img, (x, y), 2, (0, 0, 255), 2)
        cv.putText(base_img, str(i), (x, y), 
                    cv.FONT_HERSHEY_SIMPLEX, 
                    0.5, (255, 255, 0), 1)
    # Compute projected homogramphy matrix before LM
    R, t = img.compute_extrinsic(K)
    H = KRt2H(K, R, t)
    # Corners to be projected
    corners = img.corners
    corners = np.hstack((corners, np.ones((80, 1))))
    proj_world = np.dot(np.linalg.inv(H), corners.T)
    proj_corner = np.dot(H_fix, proj_world).T
    for corner in proj_corner:
        x = int(corner[0] / corner[2])
        y = int(corner[1] / corner[2])
        # Draw projected corner before LM in blue
        cv.circle(base_img, (x, y), 2, (255, 0, 0), 2)
    # Compute projected homogramphy matrix after LM
    w = ws_lm[num]
    R_lm = cv.Rodrigues(w)[0]
    t_lm = ts_lm[num]
    H_lm = KRt2H(K_lm, R_lm, t_lm)
    proj_world_lm = np.dot(np.linalg.inv(H_lm), corners.T)
    proj_corner_lm = np.dot(H_fix_lm, proj_world_lm).T
    for corner in proj_corner_lm:
        x = int(corner[0] / corner[2])
        y = int(corner[1] / corner[2])
        # Draw projected corner after LM in green
        cv.circle(base_img, (x, y), 2, (0, 255, 0), 2)

    path = img.path
    filename = f'{path.parent}\\after_LM\\{path.stem}.png'
    cv.imwrite(filename, base_img)

In [1296]:
img_num = 23
print(dataset[img_num].path.name)
R, t = dataset[img_num].compute_extrinsic(K)
np.append(R, t.reshape(-1,1), axis=1)

DSC_1587.JPG


array([[-9.799e-01, -6.656e-02,  1.879e-01,  1.360e+02],
       [ 2.281e-02, -9.738e-01, -2.261e-01,  3.625e+01],
       [ 1.980e-01, -2.172e-01,  9.558e-01, -2.922e+02]])

In [1297]:
w = ws_lm[img_num]
R_lm = cv.Rodrigues(w)[0]
t_lm = ts_lm[img_num]
np.append(R_lm, t_lm.reshape(-1,1), axis=1)

array([[-9.801e-01, -6.685e-02,  1.870e-01,  9.348e+01],
       [ 2.078e-02, -9.710e-01, -2.382e-01,  7.176e+01],
       [ 1.975e-01, -2.295e-01,  9.530e-01, -3.079e+02]])